In [1]:
import os, sys
sys.path.append("common/")

from gallery_tools import *

print "Preparing before event loop..."
mctruths_tag = art.InputTag("generator");
filename = "/Users/wketchum/MicroBooNE_Data/genie_studies/genie_uboone_10k.root"
filenames = ROOT.vector(ROOT.string)(1,filename)

# Make histograms before we open the art/ROOT file, or the file ends
# up owning the histograms.
histfile = ROOT.TFile("hist.root", "RECREATE")
npart_hist = ROOT.TH1F("npart", "Number of particles per MCTruth", 51, -0.5, 50.5)


run = array('I',[ 0 ])
event = array('I',[ 0 ])
index = array('I',[ 0 ])
n_particle = array('i',[-1])
origin = array('i',[-1])
vtx = ROOT.TLorentzVector()

mctruth_tree = ROOT.TTree("mctruth_tree","")
mctruth_tree.Branch('run',run,"run/i")
mctruth_tree.Branch('event',event,"event/i")
mctruth_tree.Branch('index',index,"index/i")
mctruth_tree.Branch('n_particle',n_particle,"n_particle/I")
mctruth_tree.Branch('origin',origin,"origin/I")
mctruth_tree.Branch('vtx',"TLorentzVector",ROOT.AddressOf(vtx))

print "Creating event object ..."
ev = gallery.Event(filenames)

# Capture the functions that will get ValidHandles. This avoids some
# inefficiency in constructing the function objects many times.
get_mctruths = ev.getValidHandle(ROOT.vector(simb.MCTruth))

print "Entering event loop..."
while (not ev.atEnd()):
    
    run[0] = ev.eventAuxiliary().run()
    event[0] = ev.eventAuxiliary().event()

    mctruths = get_mctruths(mctruths_tag)
    if (not mctruths.empty()):
        npart_hist.Fill(mctruths.product()[0].NParticles())
        index[0] = 0
        n_particle[0] = mctruths.product()[0].NParticles()
        origin[0] = mctruths.product()[0].Origin()
        vtx = mctruths.product()[0].GetNeutrino().Nu().Position()
        mctruth_tree.Fill()

    ev.next()

print "Writing histograms..."
histfile.cd()
histfile.Write()
mctruth_tree.Write()

Welcome to JupyROOT 6.12/06
Preparing before event loop...
Creating event object ...
Entering event loop...


SystemError: int TTree::Fill() =>
    problem in C++; program state has been reset